# Steps to getting an Authorization Token
1. Register a web application on MyAnimeList as described in https://myanimelist.net/blog.php?eid=835707
2. Store your MyAnimeList Client ID and Client Secret in the files `data/mal/mal_authentication/clientid.txt` and `data/mal/mal_authentication/clientsecret.txt`
3. run the notebook

In [1]:
import json
import os
import secrets

import requests

os.chdir("../../data/mal/mal_authentication")
with open("clientid.txt") as f:
    CLIENT_ID = f.read().strip()
with open("clientsecret.txt") as f:
    CLIENT_SECRET = f.read().strip()

# 1. Generate a new Code Verifier / Code Challenge.
def get_new_code_verifier() -> str:
    token = secrets.token_urlsafe(100)
    return token[:128]


# 2. Print the URL needed to authorise your application.
def print_new_authorisation_url(code_challenge: str):
    global CLIENT_ID

    url = f"https://myanimelist.net/v1/oauth2/authorize?response_type=code&client_id={CLIENT_ID}&code_challenge={code_challenge}"
    print(f"Authorise your application by clicking here: {url}\n")
    print(
        f"After pressing 'Allow', it's okay if the page fails to open, you just"
        " need to copy the URL that it tries to redirect you to\n"
    )


# 3. Once you've authorised your application, you will be redirected to the webpage you've
#    specified in the API panel. The URL will contain a parameter named "code" (the Authorisation
#    Code). You need to feed that code to the application.
def generate_new_token(authorisation_code: str, code_verifier: str) -> dict:
    global CLIENT_ID, CLIENT_SECRET

    url = "https://myanimelist.net/v1/oauth2/token"
    data = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "code": authorisation_code,
        "code_verifier": code_verifier,
        "grant_type": "authorization_code",
    }

    response = requests.post(url, data)
    response.raise_for_status()  # Check whether the requests contains errors

    token = response.json()
    response.close()
    print("Token generated successfully!")

    with open("token.json", "w") as file:
        json.dump(token, file, indent=4)
        print('Token saved in "token.json"')

    return token


# 4. Test the API by requesting your profile information
def print_user_info(access_token: str):
    url = "https://api.myanimelist.net/v2/users/@me"
    response = requests.get(url, headers={"Authorization": f"Bearer {access_token}"})

    response.raise_for_status()
    user = response.json()
    response.close()

    print(f"\n>>> Greetings {user['name']}! <<<")

In [2]:
code_verifier = code_challenge = get_new_code_verifier()
print_new_authorisation_url(code_challenge)

Authorise your application by clicking here: https://myanimelist.net/v1/oauth2/authorize?response_type=code&client_id=2c833385ced9d77b887078684b405fbd&code_challenge=_k2GmGpzjUDLXf_3tjXGMPF8EXjuDpDhrAIxKrByH-3EjJ9gc3aRugqIloVlrO6sijxmwp6TOHRvIjDGu76cuvnBUjrn91QNBQXPYrGLPa90sC-C1V_c8pxOKmy-h3ei

After pressing 'Allow', it's okay if the page fails to open, you just need to copy the URL that it tries to redirect you to



In [3]:
authorisation_code = input("Copy-paste the Authorisation Code or URL: ").strip()
default_url_prefix = "http://localhost/oauth?code="
if authorisation_code.startswith(default_url_prefix):
    authorisation_code = authorisation_code[len(default_url_prefix) :]
token = generate_new_token(authorisation_code, code_verifier)

Copy-paste the Authorisation Code or URL:  http://localhost/oauth?code=def50200661ebe620bb724db05c24b0ce89e899a24b2b2a651e3ce06b7b0abdfd1446b0e5a9f92547600c2234de1df2b9f16cb7e2684d31c1909bd6e53b6b4987fb07858b189a43b7a1c9ca5af93c031b7548346136a85befe6a8f634dfacfcca602cebaf7a04b5de3c96ceebb77797324c0eea7c99e844c96dab71cd687aa5abc8321b97ec3010be0c180531c572045414fa143d93c85920295d0679b52a3df2850431f829d8e1507de65de21dcfca6b8b64f521525d0367405b53ba06cff7351398d5e19713d8e8d6153660fed830650f354a1428753da4f89de56f3c59b0221c29095486d2f11b8e1085e09aaa11ac0431e41178efc9dcbce8e6952a412b3edf30dcba993e30fb6ff73c0815ba8e61da24aed228647ec925ddff729b474f5cdb92551265761b10b3a5b727eb8a97877227ecf63791f7cfdc08dcb918b5ce9986d2edd327a32784f24e60248dfb70215ca068828232eb67b1161c05d0698a6e3f7f4077bd00e9e67de749c4f0379f72302503540ab40cd00969a87c9c3f12580dfa8739c84bffd6dece4c56fd8dcc95c2bd62702e13cf4f8423413095f25b6dfd7fc1264fa12de68b22159bce10102d9c1fb35c3452c5822d9ad175ddc511e642e483bf694e2a85f56fc0975ac92a21

Token generated successfully!
Token saved in "token.json"


In [4]:
print_user_info(token['access_token'])


>>> Greetings Fro116! <<<
